In [1]:
from paraview.simple import *

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [2]:
import matplotlib as mpl

mpl.rcParams['axes.grid'] = True
mpl.rcParams['axes.labelsize'] = 15
mpl.rcParams['figure.labelsize'] = 'large'
mpl.rcParams['font.size'] = 15
mpl.rcParams['legend.fontsize'] = 'medium'
mpl.rcParams['legend.markerscale'] = 1.0
mpl.rcParams['lines.linewidth'] = 2
mpl.rcParams['lines.markersize'] = 4.5
savefig_directory = '../Figures/'
mpl.rcParams['xtick.labelsize'] = 15
mpl.rcParams['ytick.labelsize'] = 15

In [6]:
folder = ['src/BadWall/case.foam']

# Properties of interest with geometry storage
properties = ['Ma', 'T', 'p', 'U', 'rho']
store_files = ['Ma.png', 'T.png', 'p.png', 'U.png', 'rho.png']

In [7]:
# Load your case.foam file
foamCase = [OpenFOAMReader(FileName=folder_) for folder_ in folder]

# Update pipeline to read the time steps, etc.
[foamCase_.UpdatePipeline() for foamCase_ in foamCase]

[None]

In [8]:
foamCase_ = foamCase[0]

hasattr(foamCase_, "TimestepValues")
final_time = foamCase_.TimestepValues[-1]

In [9]:
def store_geometry(foamCase, property, store_file):
    # Create a geometry view and show the data
    renderView = GetActiveViewOrCreate('RenderView')
    SetActiveSource(foamCase)
    display = Show(foamCase, renderView)
    
    # If foamCase has multiple time steps, update to the final time step.
    if hasattr(foamCase, "TimestepValues"):# and foamCase.TimestepValues:
        # final_time = foamCase.TimestepValues[-1]
        # UpdatePipeline(time=final_time)
        final_time = foamCase_.TimestepValues[-1]
        GetAnimationScene().AnimationTime = final_time
        renderView.Update()

    # Set properties to color by the specified property
    ColorBy(display, ('POINTS', property))
    display.RescaleTransferFunctionToDataRange(True)
    display.SetScalarBarVisibility(renderView, True)

    # Set the color map for the specific property
    lut = GetColorTransferFunction(property)
    lut.ApplyPreset('Cool to Warm', True)
    display.SetRepresentationType('Surface')
    display.SetScalarBarVisibility(renderView, True)

    # Set the camera
    # Reset the camera to what would be the default in ParaView
    ResetCamera()
    # Apply camera parallel projection (orthographic view)
    renderView.CameraParallelProjection = 1
    # Show the orientation axes for better spatial reference
    renderView.OrientationAxesVisibility = 1
    # Set the background color to white
    renderView.Background = [1.0, 1.0, 1.0]
    # Set the view size
    LoadPalette('WhiteBackground')
    renderView.ViewSize = [600, 600]

    # Store a figure
    if store_file is not None:
        # Make sure directory exists before saving
        store_dir = os.path.dirname(store_file)
        if store_dir and not os.path.exists(store_dir):
            os.makedirs(store_dir, exist_ok=True)
        SaveScreenshot(store_file, renderView, magnification=2, quality=100)

    # Clean up
    Delete(display)
    Delete(renderView)

In [11]:
# Cutplots
os.makedirs('Figures', exist_ok=True)
# Loop over different geometries
for i, foamCase_ in tqdm(enumerate(foamCase)):
    # Set the active source to the current foamCase
    # Loop through each property and store the geometry
    store_file = 'Figures/BadWall'
    os.makedirs(store_file, exist_ok=True)
    # print(f'store_file: {store_file}')
    for property, store_file_ in zip(properties, store_files):
            store_file_ = f'{store_file}/{store_file_}'
            # print(f'store_file_: {store_file_}')
            store_geometry(foamCase_, property, store_file_)

1it [00:00,  1.13it/s]
